### Pytorch and Cuda installation

In [1]:
!nvcc -V

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2025 NVIDIA Corporation
Built on Fri_Feb_21_20:42:46_Pacific_Standard_Time_2025
Cuda compilation tools, release 12.8, V12.8.93
Build cuda_12.8.r12.8/compiler.35583870_0


In [2]:
# !pip3 install torch --index-url https://download.pytorch.org/whl/cu128

In [2]:
import torch
print(torch.__version__)

2.7.0+cu128


In [3]:
print(torch.cuda.is_available())

True


In [5]:
!nvidia-smi

Thu Apr 24 20:50:40 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 572.61                 Driver Version: 572.61         CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3090      WDDM  |   00000000:01:00.0  On |                  N/A |
|  0%   53C    P8             24W /  350W |    1770MiB /  24576MiB |      3%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

### Import

In [6]:
# %pip install  --upgrade \
#   "transformers==4.48.1" \
#   "datasets==3.1.0" \
#   "accelerate==1.3.0" \
#   "hf-transfer==0.1.9" \
#   "trl==0.14.0" \
#   "wandb"

In [5]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from datasets import load_dataset
from trl import PPOTrainer, PPOConfig, AutoModelForCausalLMWithValueHead
import wandb
import random
import numpy as np
from tqdm import tqdm
import tqdm as notebook_tqdm

In [ ]:
from huggingface_hub import login

login(token="", add_to_git_credential=True) # ADD YOUR TOKEN HERE

In [ ]:
dataset_id = "tau/commonsense_qa"

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from datasets import load_dataset
import torch
import random

model_id = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"

tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True
)

In [59]:
dataset = load_dataset("tau/commonsense_qa", split="validation")

# reference: https://github.com/philschmid/deep-learning-pytorch-huggingface/blob/main/training/mini-deepseek-r1-aha-grpo.ipynb
def build_r1_prompt(example):
    question = example["question"]
    choices = example["choices"]["text"]
    labels = example["choices"]["label"]
    choices_str = "\n".join([f"{label}: {text}" for label, text in zip(labels, choices)])

    r1_prefix = [
        {
            "role": "system",
            "content": "You are a helpful assistant. You first think through the reasoning, then provide the correct answer clearly in an <answer>...</answer> tag."
        },
        {
            "role": "user",
            "content": f"Given question: {question}, choose from these options {choices_str}. Show your thoughts in <think> </think> tags. And return the final answer option in <answer> </answer> tags, for example: <answer>A</answer>. "
        },
        {
            "role": "assistant",
            "content":"Let me solve this question.\n<think>"
        }
    ]

    return tokenizer.apply_chat_template(r1_prefix, tokenize=False, continue_final_message=True)
# convert our dataset to the r1 prompt
dataset = dataset.map(lambda x: {"prompt": build_r1_prompt(x)})

Map: 100%|██████████| 1221/1221 [00:00<00:00, 7630.45 examples/s]


In [60]:
dataset[0]

{'id': '1afa02df02c908a558b4036e80242fac',
 'question': 'A revolving door is convenient for two direction travel, but it also serves as a security measure at a what?',
 'question_concept': 'revolving door',
 'choices': {'label': ['A', 'B', 'C', 'D', 'E'],
  'text': ['bank', 'library', 'department store', 'mall', 'new york']},
 'answerKey': 'A',
 'prompt': '<｜begin▁of▁sentence｜>You are a helpful assistant. You first think through the reasoning, then provide the correct answer clearly in an <answer>...</answer> tag.<｜User｜>Given question: A revolving door is convenient for two direction travel, but it also serves as a security measure at a what?, choose from these options A: bank\nB: library\nC: department store\nD: mall\nE: new york. Show your thoughts in <think> </think> tags. And return the final answer option in <answer> </answer> tags, for example: <answer>A</answer>. <｜Assistant｜>Let me solve this question.\n<think>'}

In [54]:
import re

def extract_answer_tag(output_text):
    match = re.search(r"<answer>\s*([A-E])\s*</answer>", output_text)
    if match:
        return match.group(1)
    else:
        return "X"


In [ ]:
generation_kwargs = {
    "max_new_tokens": 1024,
    "do_sample": False,
    # "temperature": 0.0,
    "pad_token_id": tokenizer.eos_token_id
}

correct = 0
dataset = dataset.shuffle(seed=42).select(range(100))
total = len(dataset)

for i in notebook_tqdm.tqdm(range(len(dataset))):
    total = len(dataset)
    prompt = dataset[i]["prompt"]
    label = dataset[i]["answerKey"]

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    output_ids = model.generate(**inputs, **generation_kwargs)[0]
    output_text = tokenizer.decode(output_ids, skip_special_tokens=True)

    pred = extract_answer_tag(output_text[len(prompt):])
    # pred = output_text.split("<answer>")[-1].split("</answer>")[0].strip()
    is_correct = (pred == label)

    correct += is_correct
    print(f"\nQ{i+1}: {prompt}\n→ Predicted: {pred}, Label: {label}, {'✅' if is_correct else '❌'}")
    print(f"Answer segment: {output_text[len(prompt):].strip()}")


  0%|          | 0/100 [00:00<?, ?it/s]d:\anaconda\envs\rlproj\lib\site-packages\transformers\generation\configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
d:\anaconda\envs\rlproj\lib\site-packages\transformers\generation\configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
  1%|          | 1/100 [00:24<40:52, 24.77s/it]


Q1: <｜begin▁of▁sentence｜>You are a helpful assistant. You first think through the reasoning, then provide the correct answer clearly in an <answer>...</answer> tag.<｜User｜>Given question What are candles good for eliminating?, choose from these options A: shelf
B: board
C: church
D: table
E: dark. Show your think in <think> </think> tags. And return the final answer option in <answer> </answer> tags, for example: <answer>A</answer>. <｜Assistant｜>Let me solve this question.
<think>
→ Predicted: X, Label: E, ❌
Answer segment: ng to figure out what candles are good for eliminating. The options are A: shelf, B: board, C: church, D: table, and E: dark. Hmm, I'm not entirely sure what "eliminating" means in this context. Does it mean something like "removing" or "clearing"? Maybe it's about cleaning or something related to cleaning.

First, I'll think about what candles are typically used for. I know candles are usually used for lighting, like in homes for cooking, smoking, or as a decorati

  2%|▏         | 2/100 [00:34<26:14, 16.07s/it]


Q2: <｜begin▁of▁sentence｜>You are a helpful assistant. You first think through the reasoning, then provide the correct answer clearly in an <answer>...</answer> tag.<｜User｜>Given question Billy was an astronaut.  When he looked at the world from space, how did it look?, choose from these options A: diverse
B: round
C: square
D: orange
E: complicated. Show your think in <think> </think> tags. And return the final answer option in <answer> </answer> tags, for example: <answer>A</answer>. <｜Assistant｜>Let me solve this question.
<think>
→ Predicted: X, Label: B, ❌
Answer segment: s an astronaut, and he looked at the world from space. The question is asking, "When he looked at the world from space, how did it look?" And there are five options to choose from: A: diverse, B: round, C: square, D: orange, and E: complicated.

First, I need to understand what an astronaut sees from space. An astronaut is typically in a spacecraft, which is a large, often cylindrical structure. From space, Billy

  3%|▎         | 3/100 [00:43<20:44, 12.83s/it]

...


Q98: <｜begin▁of▁sentence｜>You are a helpful assistant. You first think through the reasoning, then provide the correct answer clearly in an <answer>...</answer> tag.<｜User｜>Given question The weasel ran up away from danger, somebody joked only our first president could get him down from the what?, choose from these options A: washington dc
B: ladder
C: natural history museum
D: cherry tree
E: chicken coop. Show your think in <think> </think> tags. And return the final answer option in <answer> </answer> tags, for example: <answer>A</answer>. <｜Assistant｜>Let me solve this question.
<think>
→ Predicted: X, Label: D, ❌
Answer segment: this question here: "The weasel ran up away from danger, somebody joked only our first president could get him down from the what?" And I need to choose the correct answer from the options A to E. Let me break this down step by step.

First, I'll read the question carefully. It says, "The weasel ran up away from danger, somebody joked only our first presid

 99%|█████████▉| 99/100 [18:50<00:12, 12.54s/it]


Q99: <｜begin▁of▁sentence｜>You are a helpful assistant. You first think through the reasoning, then provide the correct answer clearly in an <answer>...</answer> tag.<｜User｜>Given question The baby was cranky, it needed to eat breakfast but refused to what?, choose from these options A: buy food
B: open mouth
C: get out of bed
D: cry
E: wake up. Show your think in <think> </think> tags. And return the final answer option in <answer> </answer> tags, for example: <answer>A</answer>. <｜Assistant｜>Let me solve this question.
<think>
→ Predicted: A, Label: B, ❌
Answer segment: trying to figure out the answer to this question: "The baby was cranky, it needed to eat breakfast but refused to what?" The options are A: buy food, B: open mouth, C: get out of bed, D: cry, and E: wake up. 

First, I need to understand the context. The baby is cranky and needs to eat breakfast. But instead of eating, it's refusing to do something. The options given are all actions related to the baby's state or acti

100%|██████████| 100/100 [19:02<00:00, 11.42s/it]


Q100: <｜begin▁of▁sentence｜>You are a helpful assistant. You first think through the reasoning, then provide the correct answer clearly in an <answer>...</answer> tag.<｜User｜>Given question For what entity should the government work?, choose from these options A: country
B: democracy
C: canada
D: civilization
E: tax office. Show your think in <think> </think> tags. And return the final answer option in <answer> </answer> tags, for example: <answer>A</answer>. <｜Assistant｜>Let me solve this question.
<think>
→ Predicted: A, Label: A, ✅
Answer segment: to figure out the answer to the question: "For what entity should the government work?" with the options being A: country, B: democracy, C: canada, D: civilization, and E: tax office. 

First, I need to understand what the question is asking. It seems like it's asking about the role of the government in different contexts. The options are all entities, so I need to think about which one fits best in terms of government work.

Option A is "

In [58]:
print(f"Accuracy: {correct / total:.2%}")

Accuracy: 33.00%


In [62]:
# test2
generation_kwargs = {
    "max_new_tokens": 512,
    "do_sample": False,
    # "temperature": 0.0,
    "pad_token_id": tokenizer.eos_token_id
}

correct = 0
dataset = dataset.shuffle(seed=42).select(range(100))
total = len(dataset)

for i in notebook_tqdm.tqdm(range(len(dataset))):
    total = len(dataset)
    prompt = dataset[i]["prompt"]
    label = dataset[i]["answerKey"]

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    output_ids = model.generate(**inputs, **generation_kwargs)[0]
    output_text = tokenizer.decode(output_ids, skip_special_tokens=True)

    pred = extract_answer_tag(output_text[len(prompt):])
    # pred = output_text.split("<answer>")[-1].split("</answer>")[0].strip()
    is_correct = (pred == label)

    correct += is_correct
    print(f"\nQ{i+1}: {prompt}\n→ Predicted: {pred}, Label: {label}, {'✅' if is_correct else '❌'}")
    print(f"Answer segment: {output_text[len(prompt):].strip()}")


  0%|          | 0/100 [00:00<?, ?it/s]d:\anaconda\envs\rlproj\lib\site-packages\transformers\generation\configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
d:\anaconda\envs\rlproj\lib\site-packages\transformers\generation\configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
  1%|          | 1/100 [00:09<16:29, 10.00s/it]


Q1: <｜begin▁of▁sentence｜>You are a helpful assistant. You first think through the reasoning, then provide the correct answer clearly in an <answer>...</answer> tag.<｜User｜>Given question: If you are prone to postpone work what will you have to do in order to finish on time?, choose from these options A: eat
B: hasten
C: antedate
D: bring forward
E: advance. Show your thoughts in <think> </think> tags. And return the final answer option in <answer> </answer> tags, for example: <answer>A</answer>. <｜Assistant｜>Let me solve this question.
<think>
→ Predicted: C, Label: B, ❌
Answer segment: ng to figure out the answer to this question: "If you are prone to postpone work what will you have to do in order to finish on time?" The options are A: eat, B: hasten, C: antedate, D: bring forward, and E: advance. 

First, I need to understand what the question is asking. It's about procrastination and how to overcome it to finish work on time. The options seem to be related to different strategies 

  2%|▏         | 2/100 [00:20<16:39, 10.20s/it]


Q2: <｜begin▁of▁sentence｜>You are a helpful assistant. You first think through the reasoning, then provide the correct answer clearly in an <answer>...</answer> tag.<｜User｜>Given question: What is a person who is good at sports considered?, choose from these options A: lazy
B: own house
C: talented
D: affluent
E: reproduce. Show your thoughts in <think> </think> tags. And return the final answer option in <answer> </answer> tags, for example: <answer>A</answer>. <｜Assistant｜>Let me solve this question.
<think>
→ Predicted: C, Label: C, ✅
Answer segment: ng to figure out what a person who is good at sports is considered. The options are A: lazy, B: own house, C: talented, D: affluent, and E: reproduce. Hmm, okay, let's break this down.

First, I need to understand what "talented" means. Talented refers to someone who is skilled or skilled at something, often in a way that shows a high level of ability or skill. So, someone who is good at sports would likely be considered talented becaus

  3%|▎         | 3/100 [00:32<17:51, 11.05s/it]

...


Q98: <｜begin▁of▁sentence｜>You are a helpful assistant. You first think through the reasoning, then provide the correct answer clearly in an <answer>...</answer> tag.<｜User｜>Given question: Where is the first place someone leaving the planet ends up?, choose from these options A: pay debts
B: galaxy
C: outer space
D: orbit
E: universe. Show your thoughts in <think> </think> tags. And return the final answer option in <answer> </answer> tags, for example: <answer>A</answer>. <｜Assistant｜>Let me solve this question.
<think>
→ Predicted: C, Label: C, ✅
Answer segment: ng to figure out where someone who leaves the planet ends up. The options are A: pay debts, B: galaxy, C: outer space, D: orbit, and E: universe. Hmm, okay, let's break this down.

First, I need to understand what it means for someone to "leave the planet." That could mean they're escaping the planet's gravity, moving away from it, or perhaps they're leaving the planet's atmosphere. But the question is a bit vague, so I'll h

 99%|█████████▉| 99/100 [18:49<00:10, 10.58s/it]


Q99: <｜begin▁of▁sentence｜>You are a helpful assistant. You first think through the reasoning, then provide the correct answer clearly in an <answer>...</answer> tag.<｜User｜>Given question: What would be happening if you are pretending to be a police officer?, choose from these options A: fighting
B: misunderstanding
C: deception
D: play
E: distrust. Show your thoughts in <think> </think> tags. And return the final answer option in <answer> </answer> tags, for example: <answer>A</answer>. <｜Assistant｜>Let me solve this question.
<think>
→ Predicted: X, Label: C, ❌
Answer segment: to figure out what would happen if I pretend to be a police officer. The options are A: fighting, B: misunderstanding, C: deception, D: playing, and E: distrust. Hmm, let's break this down.

First, if I pretend to be a police officer, what would I actually be doing? Well, being a police officer involves responding to emergencies, maintaining order, and enforcing laws. So, if I pretend to be one, I might be try

100%|██████████| 100/100 [18:58<00:00, 11.38s/it]


Q100: <｜begin▁of▁sentence｜>You are a helpful assistant. You first think through the reasoning, then provide the correct answer clearly in an <answer>...</answer> tag.<｜User｜>Given question: Where would you find a fox that is made up?, choose from these options A: storybook
B: woods
C: hen house
D: natural habitat
E: back yard. Show your thoughts in <think> </think> tags. And return the final answer option in <answer> </answer> tags, for example: <answer>A</answer>. <｜Assistant｜>Let me solve this question.
<think>
→ Predicted: B, Label: A, ❌
Answer segment: trying to figure out where a fox made up would be found. The options are storybook, woods, hen house, natural habitat, or back yard. Hmm, let's break this down step by step.

First, I need to understand what it means for a fox to be "made up." I think it refers to a fictional or imaginative fox, not a real one. So, the question is asking where such a fictional fox would be found.

Looking at the options:

A: Storybook – That's a boo

In [63]:
print(f"Test 2 with len 512: Accuracy: {correct / total:.2%}")

Test 2 with len 512: Accuracy: 24.00%


In [69]:
# test 3: with Qwen 1.5B
model_id_qwen = "Qwen/Qwen2.5-1.5B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_id_qwen, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

model_qwen = AutoModelForCausalLM.from_pretrained(
    model_id_qwen,
    torch_dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True
)

In [72]:
dataset = load_dataset("tau/commonsense_qa", split="validation")

# reference: https://huggingface.co/Qwen/Qwen2.5-1.5B-Instruct
def build_r1_prompt(example):
    question = example["question"]
    choices = example["choices"]["text"]
    labels = example["choices"]["label"]
    choices_str = "\n".join([f"{label}: {text}" for label, text in zip(labels, choices)])

    r1_prefix = [
        {
            "role": "system",
            "content": "You are a helpful assistant. You first think through the reasoning, then provide the correct answer clearly in an <answer>...</answer> tag."
        },
        {
            "role": "user",
            "content": f"Given question: {question}, choose from these options {choices_str}. Think step by step, and return the final answer option in <answer> </answer> tags, for example: <answer>A</answer>. "
        }
    ]

    return tokenizer.apply_chat_template(r1_prefix, tokenize=False, continue_final_message=True)
# convert our dataset to the r1 prompt
dataset_qwen = dataset.map(lambda x: {"prompt": build_r1_prompt(x)})

Map: 100%|██████████| 1221/1221 [00:00<00:00, 8944.55 examples/s]


In [73]:
generation_kwargs = {
    "max_new_tokens": 1024,
    "do_sample": False,
    # "temperature": 0.0,
    "pad_token_id": tokenizer.eos_token_id
}

correct = 0
dataset_qwen = dataset_qwen.shuffle(seed=42).select(range(100))
total = len(dataset_qwen)

for i in notebook_tqdm.tqdm(range(len(dataset_qwen))):
    total = len(dataset_qwen)
    prompt = dataset_qwen[i]["prompt"]
    label = dataset_qwen[i]["answerKey"]

    inputs = tokenizer(prompt, return_tensors="pt").to(model_qwen.device)
    output_ids = model_qwen.generate(**inputs, **generation_kwargs)[0]
    output_text = tokenizer.decode(output_ids, skip_special_tokens=True)

    pred = extract_answer_tag(output_text[len(prompt):])
    # pred = output_text.split("<answer>")[-1].split("</answer>")[0].strip()
    is_correct = (pred == label)

    correct += is_correct
    print(f"\nQ{i+1}: {prompt}\n→ Predicted: {pred}, Label: {label}, {'✅' if is_correct else '❌'}")
    print(f"Answer segment: {output_text[len(prompt):].strip()}")


  0%|          | 0/100 [00:00<?, ?it/s]d:\anaconda\envs\rlproj\lib\site-packages\transformers\generation\configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
d:\anaconda\envs\rlproj\lib\site-packages\transformers\generation\configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.8` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
d:\anaconda\envs\rlproj\lib\site-packages\transformers\generation\configuration_utils.py:650: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `20` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(
  1%|          | 1/100 [00:05<09:34,  5.80s/it]


Q1: <|im_start|>system
You are a helpful assistant. You first think through the reasoning, then provide the correct answer clearly in an <answer>...</answer> tag.<|im_end|>
<|im_start|>user
Given question: If you are prone to postpone work what will you have to do in order to finish on time?, choose from these options A: eat
B: hasten
C: antedate
D: bring forward
E: advance. Think step by step, and return the final answer option in <answer> </answer> tags, for example: <answer>A</answer>. 
→ Predicted: D, Label: B, ❌
Answer segment: eone who is prone to postponing their work.
2. The goal is to find out what action they need to take to complete their tasks on time.
3. Let's analyze each option:
   - Option A: "eat" doesn't directly relate to completing work or being timely.
   - Option B: "hasten" means to speed up or hurry, which could help but isn't necessarily effective for all situations.
   - Option C: "antedate" typically refers to adding extra days to a contract or agreement, no

  2%|▏         | 2/100 [00:10<08:31,  5.22s/it]


Q2: <|im_start|>system
You are a helpful assistant. You first think through the reasoning, then provide the correct answer clearly in an <answer>...</answer> tag.<|im_end|>
<|im_start|>user
Given question: What is a person who is good at sports considered?, choose from these options A: lazy
B: own house
C: talented
D: affluent
E: reproduce. Think step by step, and return the final answer option in <answer> </answer> tags, for example: <answer>A</answer>. 
→ Predicted: C, Label: C, ✅
Answer segment: n who is good at sports.
2. We need to analyze each option:
   - Option A: "lazy" - This does not describe someone skilled or talented in sports.
   - Option B: "own house" - While owning a house can be associated with affluence, it doesn't directly relate to being good at sports.
   - Option C: "talented" - This describes having natural ability or skill in something, which fits well with being good at sports.
   - Option D: "affluent" - Affluence refers to wealth or financial status, but i

  3%|▎         | 3/100 [00:14<07:25,  4.59s/it]

...


Q98: <|im_start|>system
You are a helpful assistant. You first think through the reasoning, then provide the correct answer clearly in an <answer>...</answer> tag.<|im_end|>
<|im_start|>user
Given question: Where is the first place someone leaving the planet ends up?, choose from these options A: pay debts
B: galaxy
C: outer space
D: orbit
E: universe. Think step by step, and return the final answer option in <answer> </answer> tags, for example: <answer>A</answer>. 
→ Predicted: X, Label: C, ❌
Answer segment: into orbit around it.
2) Once in orbit, they can move to other planets or moons.
3) The farthest anyone has ever gone away from Earth is to the moon.
4) So when you ask where does something end up after leaving earth, the answer would be the moon.
5) The moon is not part of our solar system but rather its own separate celestial body.
6) Therefore, the last thing that leaves Earth before reaching the moon is orbit.
<answer>orbit</answer>


 99%|█████████▉| 99/100 [07:37<00:03,  3.38s/it]


Q99: <|im_start|>system
You are a helpful assistant. You first think through the reasoning, then provide the correct answer clearly in an <answer>...</answer> tag.<|im_end|>
<|im_start|>user
Given question: What would be happening if you are pretending to be a police officer?, choose from these options A: fighting
B: misunderstanding
C: deception
D: play
E: distrust. Think step by step, and return the final answer option in <answer> </answer> tags, for example: <answer>A</answer>. 
→ Predicted: C, Label: C, ✅
Answer segment: olice officer, they are engaging in a form of deception.
2. Deception involves presenting false information or disguising oneself as something one is not.
3. The primary purpose of pretending to be a police officer is often to deceive others into thinking that person is actually a law enforcement officer.
4. Therefore, among the given options, "deception" best describes what would be happening when someone pretends to be a police officer.
5. None of the other opti

100%|██████████| 100/100 [07:41<00:00,  4.62s/it]


Q100: <|im_start|>system
You are a helpful assistant. You first think through the reasoning, then provide the correct answer clearly in an <answer>...</answer> tag.<|im_end|>
<|im_start|>user
Given question: Where would you find a fox that is made up?, choose from these options A: storybook
B: woods
C: hen house
D: natural habitat
E: back yard. Think step by step, and return the final answer option in <answer> </answer> tags, for example: <answer>A</answer>. 
→ Predicted: A, Label: A, ✅
Answer segment: y found in their natural habitats such as forests, grasslands, or deserts.
2. Storybooks often feature fictional characters like foxes, but they don't contain real foxes.
3. Woods can be home to foxes, making it a plausible location.
4. Hen houses are not associated with foxes; they're more commonly known for containing chickens.
5. Natural habitats of foxes include forests, grasslands, and deserts, which might have some foxes present.
6. Backyards could potentially have foxes if someon

In [74]:
print(f"Test 3 on Qwen 2.5 1.5b with len 1024: Accuracy: {correct / total:.2%}")

Test 3 on Qwen 2.5 1.5b with len 1024: Accuracy: 59.00%


In [ ]:
dataset = load_dataset("tau/commonsense_qa", split="validation")

# reference: https://github.com/philschmid/deep-learning-pytorch-huggingface/blob/main/training/mini-deepseek-r1-aha-grpo.ipynb
# gemerate r1 prompt with a prefix for the model to already start with the thinking process
def build_r1_prompt(example):
    question = example["question"]
    choices = example["choices"]["text"]
    labels = example["choices"]["label"]
    choices_str = "\n".join([f"{label}: {text}" for label, text in zip(labels, choices)])

    # mimic 
    r1_prefix = [
        {
            "role": "system",
            "content": "You are a helpful assistant. You first think through the reasoning, then provide the correct answer clearly in an <answer>...</answer> tag."
        },
        {
            "role": "user",
            "content": f"Given question: {question}, choose from these options {choices_str}. Show your thoughts in <think> </think> tags. And return the final answer option in <answer> </answer> tags, for example: <answer>A</answer>. "
        },
        {
            "role": "assistant",
            "content":"Let me solve this question.\n<think>"
        }
    ]

    return tokenizer.apply_chat_template(r1_prefix, tokenize=False, continue_final_message=True)
# convert our dataset to the r1 prompt
dataset_qwen = dataset.map(lambda x: {"prompt": build_r1_prompt(x)})

In [ ]:
generation_kwargs = {
    "max_new_tokens": 512,
    "do_sample": False,
    # "temperature": 0.0,
    "pad_token_id": tokenizer.eos_token_id
}

correct = 0
dataset_qwen = dataset_qwen.shuffle(seed=42).select(range(100))
total = len(dataset_qwen)

for i in notebook_tqdm.tqdm(range(len(dataset_qwen))):
    total = len(dataset_qwen)
    prompt = dataset_qwen[i]["prompt"]
    label = dataset_qwen[i]["answerKey"]

    inputs = tokenizer(prompt, return_tensors="pt").to(model_qwen.device)
    output_ids = model_qwen.generate(**inputs, **generation_kwargs)[0]
    output_text = tokenizer.decode(output_ids, skip_special_tokens=True)

    pred = extract_answer_tag(output_text[len(prompt):])
    # pred = output_text.split("<answer>")[-1].split("</answer>")[0].strip()
    is_correct = (pred == label)

    correct += is_correct
    print(f"\nQ{i+1}: {prompt}\n→ Predicted: {pred}, Label: {label}, {'✅' if is_correct else '❌'}")
    print(f"Answer segment: {output_text[len(prompt):].strip()}")
